In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cosine, correlation


In [2]:
ratings=pd.read_csv('Data/ratings.csv')
links=pd.read_csv('Data/links.csv')
tags=pd.read_csv('Data/tags.csv')
movies=pd.read_csv('Data/movies.csv')

ratings

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [3]:
tags

,userId,movieId,tag,timestamp
0,2,60756,funny,1445714994
1,2,60756,Highly quotable,1445714996
2,2,60756,will ferrell,1445714992
3,2,89774,Boxing story,1445715207
4,2,89774,MMA,1445715200
...,...,...,...,...
3678,606,7382,for katie,1171234019
3679,606,7936,austere,1173392334
3680,610,3265,gun fu,1493843984
3681,610,3265,heroic bloodshed,1493843978


In [4]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [5]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [6]:
links

,movieId,imdbId,tmdbId
0,1,114709,862.0
1,2,113497,8844.0
2,3,113228,15602.0
3,4,114885,31357.0
4,5,113041,11862.0
...,...,...,...
9737,193581,5476944,432131.0
9738,193583,5914996,445030.0
9739,193585,6397426,479308.0
9740,193587,8391976,483455.0


In [7]:
#join the tables together

df_joined= pd.merge(movies, ratings, on='movieId')

In [8]:
df_joined
df_new = df_joined.drop(['genres', 'title','timestamp'], axis=1)

In [9]:
from surprise import Reader, Dataset
reader = Reader(rating_scale=(0.5, 5.0))
data = Dataset.load_from_df(ratings[['movieId', 'userId', 'rating']],reader)
# data = Dataset.load_builtin('ml-100k')

In [10]:
dataset = data.build_full_trainset()

In [11]:
from surprise.model_selection import cross_validate
from surprise.prediction_algorithms import SVD
from surprise.prediction_algorithms import KNNWithMeans, KNNBasic, KNNBaseline
from surprise.model_selection import GridSearchCV
import numpy as np

In [12]:
# #need to add more to the grid search later
# params = {'n_factors': [20, 50, 100],
#          'reg_all': [0.02, 0.05, 0.1]}
# g_s_svd = GridSearchCV(SVD,param_grid=params,n_jobs=-1)
# g_s_svd.fit(data)

# print(g_s_svd.best_score)
# print(g_s_svd.best_params)

### using KNN


In [13]:
# knn_baseline = KNNBaseline(sim_options={'name':'pearson', 'user_based':True})
# cv_knn_baseline = cross_validate(knn_baseline,data)


In [14]:
# for i in cv_knn_baseline.items():
#     print(i)

# np.mean(cv_knn_baseline['test_rmse'])

NameError: name 'cv_knn_baseline' is not defined

In [ ]:
svd = SVD(n_factors= 50, reg_all=0.05)
svd.fit(dataset)

In [ ]:
svd.predict(2, 4)

In [15]:
def movie_rater(movie_df,num, genre=None):
    userID = 1000
    rating_list = []
    while num > 0:
        if genre:
            movie = movie_df[movie_df['genres'].str.contains(genre)].sample(1)
        else:
            movie = movie_df.sample(1)
        print(movie)
        rating = input('How do you rate this movie on a scale of 1-5, press n if you have not seen :\n')
        if rating == 'n':
            continue
        else:
            rating_one_movie = {'userId':userID,'movieId':movie['movieId'].values[0],'rating':rating}
            rating_list.append(rating_one_movie) 
            num -= 1
    return rating_list      

In [16]:
df_joined.drop(['userId', 'timestamp','rating'], axis=1)

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...
100831,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
100832,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
100833,193585,Flint (2017),Drama
100834,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [53]:
user_rating = movie_rater(df_joined, 3, 'Action')

      movieId                              title                 genres  \
4922      165  Die Hard: With a Vengeance (1995)  Action|Crime|Thriller   

      userId  rating   timestamp  
4922     517     2.5  1487957639  
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3
       movieId                                  title  \
32624     1544  Lost World: Jurassic Park, The (1997)   

                                 genres  userId  rating   timestamp  
32624  Action|Adventure|Sci-Fi|Thriller     608     3.5  1117504496  
How do you rate this movie on a scale of 1-5, press n if you have not seen :
3
      movieId                   title                  genres  userId  rating  \
5104      172  Johnny Mnemonic (1995)  Action|Sci-Fi|Thriller     604     2.0   

      timestamp  
5104  832080293  
How do you rate this movie on a scale of 1-5, press n if you have not seen :
4


In [54]:
user_rating

[{'userId': 1000, 'movieId': 165, 'rating': '3'},
 {'userId': 1000, 'movieId': 1544, 'rating': '3'},
 {'userId': 1000, 'movieId': 172, 'rating': '4'}]

In [55]:

new_ratings_df = df_new.append(user_rating,ignore_index=True)
new_data = Dataset.load_from_df(new_ratings_df,reader)
    

In [56]:
svd_ = SVD(n_factors= 50, reg_all=0.05)
svd_.fit(new_data.build_full_trainset())

In [ ]:
list_of_movies = []
for m_id in df_new['movieId'].unique():
    list_of_movies.append( (m_id,svd_.predict(100,m_id)[3]))

In [ ]:
svd_.predict(150,3)

In [ ]:
list_of_movies

In [ ]:
ranked_movies = sorted(list_of_movies, key=lambda x:x[1], reverse=True)

In [ ]:
ranked_movies

In [ ]:
def recommended_movies(user_ratings,movie_title_df,n):
        for idx, rec in enumerate(user_ratings):
            title = movie_title_df.loc[movie_title_df['movieId'] == int(rec[0])]['title']
            print('Recommendation # ', idx+1, ': ', title, '\n')
            n-= 1
            if n == 0:
                break
            
recommended_movies(ranked_movies,movies,4)

## Content Based Filtering

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
# Define a TF-IDF Vectorizer Object.
tfidf_movies_genres = TfidfVectorizer(token_pattern = '[a-zA-Z0-9\-]+')

In [22]:
tfidf_movies_genres_matrix = tfidf_movies_genres.fit_transform(movies['genres'])
cosine_score = linear_kernel(tfidf_movies_genres_matrix, tfidf_movies_genres_matrix)

In [23]:
#using TF-IDF Vectorizer Oject to calculate recommendations
def get_recommendations_based_on_genres(movie_title, cosine_score=cosine_score):
    """
    Calculates top 2 movies to recommend based on given movie titles genres. 
    :param movie_title: title of movie to be taken for base of recommendation
    :param cosine_sim_movies: cosine similarity between movies 
    :return: Titles of movies recommended to user
    """
    # Get the index of the movie that matches the title
    idx_movie = movies.loc[movies['title'].isin([movie_title])]
    idx_movie = idx_movie.index
    
    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores_movies = list(enumerate(cosine_score[idx_movie][0]))
    
    # Sort the movies based on the similarity scores
    sim_scores_movies = sorted(sim_scores_movies, key=lambda x: x[1], reverse=True)

    # Get the scores of the 10 most similar movies
    sim_scores_movies = sim_scores_movies[0:4]
    
    
    # Get the movie indices
    movie_indices = [i[0] for i in sim_scores_movies]
    
    # Return the top 2 most similar movies
    return movies['title'].iloc[movie_indices]
    
    
#     movie_indices = [movies['title'].iloc[i[0]] for i in sim_scores_movies if movies['title'].iloc[i[0]]!=movie_title ]
    
#     # Return the top 2 most similar movies
#     print(movie_indices)



In [24]:
#robocop why does it recommend robots
get_recommendations_based_on_genres('Steamboat Willie (1928)')

1543                       Jungle Book, The (1967)
1564                       Steamboat Willie (1928)
3727         Ferngully: The Last Rainforest (1992)
5232    Claymation Christmas Celebration, A (1987)
Name: title, dtype: object

In [25]:
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [48]:
def get_recommendation_content_model(userId):
    """
    Calculates top movies to be recommended to user based on movie user has watched.  
    :param userId: userid of user
    :return: Titles of movies recommended to user
    """
    recommended_movie_list = []
    movie_list = []
    df_rating_filtered = ratings[ratings["userId"]== userId]
    for key, row in df_rating_filtered.iterrows():
      
        movie_list.append((movies["title"][row["movieId"]==movies["movieId"]]).values) 
    for index, movie in enumerate(movie_list):
        for key, movie_recommended in get_recommendations_based_on_genres(movie[0]).iteritems():
            recommended_movie_list.append(movie_recommended)

    # removing already watched movie from recommended list    
    for movie_title in recommended_movie_list:
        if movie_title in movie_list:
            recommended_movie_list.remove(movie_title)
    
    return set(recommended_movie_list)
get_recommendation_content_model(2)

{'39 Steps, The (1935)',
 'Abyss, The (1989)',
 'Ace Ventura: When Nature Calls (1995)',
 'All Quiet on the Western Front (1930)',
 'Amateur (1994)',
 'Anne Frank Remembered (1995)',
 'Apocalypse Now (1979)',
 'Assassins (1995)',
 'Babysitter, The (1995)',
 'Batman Begins (2005)',
 'Ben-Hur (1959)',
 'Beverly Hills Ninja (1997)',
 'Big One, The (1997)',
 'Bio-Dome (1996)',
 'Boot, Das (Boat, The) (1981)',
 'Braveheart (1995)',
 'Buffy the Vampire Slayer (1992)',
 'Butch Cassidy and the Sundance Kid (1969)',
 'Casino (1995)',
 'Catwalk (1996)',
 'City Hall (1996)',
 'Clear and Present Danger (1994)',
 'Client, The (1994)',
 'Cloud Atlas (2012)',
 'Contagion (2011)',
 'Crossing Guard, The (1995)',
 'Cry, the Beloved Country (1995)',
 'Dangerous Minds (1995)',
 'Day the Earth Stood Still, The (1951)',
 'Dead Man Walking (1995)',
 'Dead Snow 2: Red vs. Dead (2014) ',
 'Deep Impact (1998)',
 'Die Hard: With a Vengeance (1995)',
 'Doom Generation, The (1995)',
 'Escape from L.A. (1996)',
 'E

In [45]:
movies

,movieId,title,genre,similarity
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.359809
1,2,Jumanji (1995),Adventure|Children|Fantasy,0.304119
2,3,Grumpier Old Men (1995),Comedy|Romance,0.158762
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0.026097
4,5,Father of the Bride Part II (1995),Comedy,0.132805
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,NaN
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,NaN
9739,193585,Flint (2017),Drama,NaN
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,NaN


### Item to Item Rating

#### df_joined

In [27]:
df_movies_ratings=pd.merge(movies, ratings)
ratings_matrix_items = df_movies_ratings.pivot_table(index=['movieId'],columns=['userId'],values='rating').reset_index(drop=True)
ratings_matrix_items.fillna( 0, inplace = True )


In [28]:
ratings_matrix_items

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
0,4.0,0.0,0.0,0.0,4.0,0.0,4.5,0.0,0.0,0.0,...,4.0,0.0,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
1,0.0,0.0,0.0,0.0,0.0,4.0,0.0,4.0,0.0,0.0,...,0.0,4.0,0.0,5.0,3.5,0.0,0.0,2.0,0.0,0.0
2,4.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9719,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9720,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
9722,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [29]:
movie_similarity = 1 - pairwise_distances( ratings_matrix_items.values, metric="cosine" )
np.fill_diagonal( movie_similarity, 0 ) #Filling diagonals with 0s for future use when sorting is done
ratings_matrix_items = pd.DataFrame( movie_similarity )
ratings_matrix_items

,0,1,2,3,4,5,6,7,8,9,...,9714,9715,9716,9717,9718,9719,9720,9721,9722,9723
0,0.000000,0.410562,0.296917,0.035573,0.308762,0.376316,0.277491,0.131629,0.232586,0.395573,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.410562,0.000000,0.282438,0.106415,0.287795,0.297009,0.228576,0.172498,0.044835,0.417693,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.296917,0.282438,0.000000,0.092406,0.417802,0.284257,0.402831,0.313434,0.304840,0.242954,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.035573,0.106415,0.092406,0.000000,0.188376,0.089685,0.275035,0.158022,0.000000,0.095598,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.308762,0.287795,0.417802,0.188376,0.000000,0.298969,0.474002,0.283523,0.335058,0.218061,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9719,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0
9720,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
9721,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,0.0
9722,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0


In [41]:
#function where returns similar movies based on item to item:

def item_similarity(movieName): 
    """
    recomendates similar movies
   :param data: name of the movie 
   """
    try:
        #user_inp=input('Enter the reference movie title based on which recommendations are to be made: ')
        user_inp=movieName
        inp=movies[movies['title']==user_inp].index.tolist()
        inp=inp[0]

        movies['similarity'] = ratings_matrix_items.iloc[inp]
        movies.columns = ['movieId', 'title', 'genre','similarity']
    except:
        print("Sorry, the movie is not in the database!")

In [44]:
item_similarity('Jungle Book, The (1967)')

In [32]:
df_joined.loc[df_joined.userId == 54]

,movieId,title,genres,userId,rating,timestamp
18,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,54,3.0,830247330
960,17,Sense and Sensibility (1995),Drama|Romance,54,3.0,830248246
1157,21,Get Shorty (1995),Comedy|Crime|Thriller,54,3.0,839921390
1517,32,Twelve Monkeys (a.k.a. 12 Monkeys) (1995),Mystery|Sci-Fi|Thriller,54,3.0,830247417
2125,47,Seven (a.k.a. Se7en) (1995),Mystery|Thriller,54,3.0,839921084
2400,50,"Usual Suspects, The (1995)",Crime|Mystery|Thriller,54,3.0,830248246
3406,110,Braveheart (1995),Action|Drama|War,54,3.0,830247832
4072,150,Apollo 13 (1995),Adventure|Drama|IMAX,54,3.0,830247106
4610,161,Crimson Tide (1995),Drama|Thriller|War,54,3.0,830247358
5505,194,Smoke (1995),Comedy|Drama,54,3.0,830247865


In [78]:
def recommendedMoviesAsperItemSimilarity(user_id):
    """
     Recommending movie which user hasn't watched as per Item Similarity
    :param user_id: user_id to whom movie needs to be recommended
    :return: movieIds to user 
    """
    
    try:
        user_movie= df_movies_ratings[(df_movies_ratings.userId==user_id) & df_movies_ratings.rating.isin(range(4,5))][['title']]
        user_movie=user_movie.iloc[0,0]

        item_similarity(user_movie)

        sorted_movies_as_per_userChoice=movies.sort_values( ["similarity"], ascending = False )
        sorted_movies_as_per_userChoice=sorted_movies_as_per_userChoice[sorted_movies_as_per_userChoice['similarity'] >=0.45]['movie_id']
        recommended_movies=list()
        df_recommended_item=pd.DataFrame()
        user2Movies= ratings[ratings['userId']== user_id]['movieId']
        for movieId in sorted_movies_as_per_userChoice:
                if movieId not in user2Movies:
                    d=[]

                    df_new= ratings[(ratings.movieId==movieId)]
                    df_recommended_item=pd.concat([df_recommended_item,df_new])
                    best10=df_recommended_item.sort_values(["rating"], ascending = False )[1:25] 
                    d= set(best10['movieId'].tolist())


        return d
    
    
    #recommend based on disliked movies if the selected user does not have a 4 or above rated movie
    except:
        print("this user does not have a movie she has rated above 4 to start the recommendation")
        user_movie= df_movies_ratings[(df_movies_ratings.userId==user_id) & df_movies_ratings.rating.isin(range(0,3))][['title']]
        user_movie=user_movie.iloc[0,0]

        item_similarity(user_movie)

        sorted_movies_as_per_userChoice=movies.sort_values( ["similarity"], ascending = True )
        sorted_movies_as_per_userChoice=sorted_movies_as_per_userChoice[sorted_movies_as_per_userChoice['similarity'] < 0.1]['movieId']
        recommended_movies=list()
        df_recommended_item=pd.DataFrame()
        user2Movies= ratings[ratings['userId']== user_id]['movieId']
        for movieId in sorted_movies_as_per_userChoice:
                if movieId not in user2Movies:
                    d=[]

                    df_new= ratings[(ratings.movieId==movieId)]
                    df_recommended_item=pd.concat([df_recommended_item,df_new])
                    best10=df_recommended_item.sort_values(["rating"], ascending = False )[1:10] 
                    d= set(best10['movieId'].tolist())


        return d

In [79]:
recommendedMoviesAsperItemSimilarity(54)

this user does not have a movie she has rated above 4 to start the recommendation


{838, 1084, 1206}

In [81]:
def movieIdToTitle(listMovieIDs):
    """
     Converting movieId to titles
    :param user_id: List of movies
    :return: movie titles
    """
    movie_titles= list()
    for id in listMovieIDs:
        movie_titles.append(movies[movies['movieId']==id]['title'])
    return movie_titles

In [82]:
movieIdToTitle(recommendedMoviesAsperItemSimilarity(54))

this user does not have a movie she has rated above 4 to start the recommendation


[650    Emma (1996)
 Name: title, dtype: object,
 824    Bonnie and Clyde (1967)
 Name: title, dtype: object,
 907    Clockwork Orange, A (1971)
 Name: title, dtype: object]

In [75]:
movies

,movieId,title,genre,similarity
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,0.277491
1,2,Jumanji (1995),Adventure|Children|Fantasy,0.228576
2,3,Grumpier Old Men (1995),Comedy|Romance,0.402831
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance,0.275035
4,5,Father of the Bride Part II (1995),Comedy,0.474002
...,...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy,NaN
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy,NaN
9739,193585,Flint (2017),Drama,NaN
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation,NaN


#### Hybrid Model

In [83]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

svd = SVD()
trainset = data.build_full_trainset()
svd.fit(trainset)

cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8749  0.8733  0.8748  0.8725  0.8721  0.8735  0.0012  
MAE (testset)     0.6707  0.6718  0.6734  0.6688  0.6688  0.6707  0.0018  
Fit time          3.78    3.83    3.79    3.77    3.78    3.79    0.02    
Test time         0.08    0.08    0.08    0.08    0.08    0.08    0.00    


{'test_rmse': array([0.87490001, 0.87329038, 0.87479689, 0.87245353, 0.87209193]),
 'test_mae': array([0.67072163, 0.67184167, 0.67336004, 0.66879945, 0.66880471]),
 'fit_time': (3.7781221866607666,
  3.8328561782836914,
  3.788116931915283,
  3.7705490589141846,
  3.780202865600586),
 'test_time': (0.08362603187561035,
  0.08196902275085449,
  0.08222699165344238,
  0.08065009117126465,
  0.08208227157592773)}

In [76]:
def hybrid_model(user_id):
    recommended_movies_by_content_model = get_recommendation_content_model(user_id)
    recommended_movies_by_content_model = movies[movies.apply(lambda movie: movie["title"] in recommended_movies_by_content_model, axis=1)]
                                      
    for key, col in recommended_movies_by_content_model.iterrows():
        predict = svd.predict(user_id, col["movieId"])
        recommended_movies_by_content_model.loc[key, "svd_rating"] = predict.est
    
    
    return recommended_movies_by_content_model.sort_values("svd_rating", ascending=False).iloc[0:11]

hybrid_model(33)

/Users/tbai/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/indexing.py:1596: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)
/Users/tbai/opt/anaconda3/envs/learn-env/lib/python3.8/site-packages/pandas/core/indexing.py:1765: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value)


,movieId,title,genre,similarity,svd_rating
224,260,Star Wars: Episode IV - A New Hope (1977),Action|Adventure|Sci-Fi,0.217562,4.479477
909,1208,Apocalypse Now (1979),Action|Drama|War,0.048022,4.410043
97,110,Braveheart (1995),Action|Drama|War,0.252985,4.368797
951,1252,Chinatown (1974),Crime|Film-Noir|Mystery|Thriller,0.068832,4.334396
27,28,Persuasion (1995),Drama|Romance,0.128464,4.269242
828,1089,Reservoir Dogs (1992),Crime|Mystery|Thriller,0.117380,4.257878
933,1233,"Boot, Das (Boat, The) (1981)",Action|Drama|War,0.087837,4.249285
1640,2186,Strangers on a Train (1951),Crime|Drama|Film-Noir|Thriller,0.158970,4.197847
692,910,Some Like It Hot (1959),Comedy|Crime,0.164174,4.197601
661,866,Bound (1996),Crime|Drama|Romance|Thriller,0.048134,4.183272


## SVD ++

In [85]:
from surprise.model_selection import train_test_split

data = Dataset.load_builtin('ml-100k')
trainset, testset = train_test_split(data, test_size=.2)

In [88]:
from surprise import SVDpp
from surprise import accuracy

In [89]:
algo_svdpp = SVDpp(n_factors=150, n_epochs=8, lr_all=0.005, reg_all=0.1)
algo_svdpp.fit(trainset)
test_pred = algo_svdpp.test(testset)
print("SVDpp : Test Set")
accuracy.rmse(test_pred, verbose=True)

SVDpp : Test Set
RMSE: 0.9468


0.9468183423713388